<a href="https://colab.research.google.com/github/sanazy/Plate-Detection-Project/blob/master/05_traffic_freeze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
EFF_PATH = '/content/drive/My\ Drive/Traffic/EfficientDet'
%cd {EFF_PATH}

/content/drive/My Drive/Traffic/EfficientDet


In [ ]:
!pip install -r {EFF_PATH}/requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-6jeunuwf
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-6jeunuwf
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266988 sha256=f4afb0b1ce0dca49790ada539b37f8ce9fc8f1a1f1dc560f0438cd6b8fb7884e
  Stored in directory: /tmp/pip-ephem-wheel-cache-yhb1yza5/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools


In [ ]:
from model import efficientdet
import cv2
import os
import numpy as np
import time
from utils import preprocess_image
import tensorflow as tf
from tensorflow.keras import backend as K

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
  graph = session.graph
  with graph.as_default():
    freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
    output_names = output_names or []
    output_names += [v.op.name for v in tf.global_variables()]
    input_graph_def = graph.as_graph_def()
    if clear_devices:
      for node in input_graph_def.node:
        node.device = ""
    frozen_graph = tf.graph_util.convert_variables_to_constants(
        session, input_graph_def, output_names, freeze_var_names)
    return frozen_graph

In [ ]:
def freeze_model():
  phi = 0
  weighted_bifpn = False
  model_path = 'checkpoints/2020-07-02-13:29:19-10-5-900/csv_10_0.1043_0.1445.h5'
  #model_path = 'checkpoints/2020-06-21-07:14:54-10-5-900/csv_10_0.1047_0.1514.h5'
  image_sizes = (256, 640, 768, 896, 1024, 1280, 1408)
  image_size = image_sizes[phi]
  classes = ['light', 'heavy', 'unknown']
  num_classes = len(classes)
  score_threshold = 0.5
  model, prediction_model = efficientdet(phi=phi,
                                          weighted_bifpn=weighted_bifpn,
                                          num_classes=num_classes,
                                          score_threshold=score_threshold)
  prediction_model.load_weights(model_path, by_name=True)
  
  frozen_graph = freeze_session(K.get_session(),  output_names=[out.op.name for out in prediction_model.outputs])
  tf.train.write_graph(frozen_graph, "./checkpoints/", "2020-07-02.pb", as_text=False)

In [ ]:
freeze_model()

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 652 variables.
INFO:tensorflow:Converted 652 variables to const ops.
